# Initialization


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
%cd PhysGNN

# Package Installation

In [ ]:
!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://pytorch-geometric.com/whl/torch-1.9.0+cu102.html
!pip install tensorboardX
!pip install networkx

# Importing the libraries

In [ ]:
import random 
from random import shuffle
import math
import statistics

import tensorflow as tf

import os
from os.path import exists

import datetime
import pickle 
from joblib import dump, load

import numpy as np
import networkx as nx

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import Sequential, Linear, ReLU, GRU
from torch_geometric.nn import JumpingKnowledge

import torch_geometric as tg
import torch_geometric.nn as tg_nn
import torch_geometric.utils as tg_utils
from torch_geometric.nn import MessagePassing
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.nn import Sequential, JumpingKnowledge
from torch_geometric.transforms import NormalizeFeatures

from torch.nn import init
import pdb

from torch_geometric.data import Data, DataLoader

import torch.optim as optim
import torch_geometric.transforms as T

import torchvision
import torchvision.transforms as transforms

import time
from datetime import datetime

import multiprocessing as mp

# For visualizing the results
from tensorboardX import SummaryWriter
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt

import pandas as pd
from torch_geometric.utils.convert import to_networkx

print(torch.__version__)
print(torchvision.__version__)

# Models


##Config 1


In [ ]:
# ----------------------------- Configuration 1 --------------------------------
# -----------------------------                  --------------------------------
class config1(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config1, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='max')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

## Config2

In [ ]:
# ----------------------------- Configuration 2 --------------------------------
# -----------------------------                  --------------------------------
class config2(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config2, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config3


In [ ]:
# ----------------------------- Configuration 3 ---------------------------------
# -----------------------------                  --------------------------------
class config3(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config3, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config4

In [ ]:
# ----------------------------- Configuration 4 --------------------------------
# -----------------------------                  --------------------------------
class config4(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config4, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config5

In [ ]:
# ----------------------------- Configuration 5 --------------------------------
# -----------------------------                  --------------------------------
class config5(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config5, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config 6

In [ ]:
# ----------------------------- Configuration 6 --------------------------------
# -----------------------------                  --------------------------------
class config6(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config6, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config 7

In [ ]:
# ----------------------------- Configuration 7 --------------------------------
# -----------------------------                  --------------------------------
class config7(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config7, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config8

In [ ]:
# ----------------------------- Configuration 8 --------------------------------
# -----------------------------                  --------------------------------
class config8(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config8, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')
        
        if dataset_name == 'dataset_1':
            self.conv2 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')  
        else:
            self.conv2 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='add')
            
        self.conv3 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config9

In [ ]:
# ----------------------------- Configuration 9 --------------------------------
# -----------------------------                  --------------------------------
class config9(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config9, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')

        if dataset_name == 'dataset_1':
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')   
        else:
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
            
        self.conv3 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config 10

In [ ]:
# ----------------------------- Configuration 10 --------------------------------
# -----------------------------                  --------------------------------
class config10(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config10, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')

        if dataset_name == 'dataset_1':       
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        else:
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
              
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index, edge_weight)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config 11

In [ ]:
# ----------------------------- Configuration 11 --------------------------------
# -----------------------------                  --------------------------------
class config11(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config11, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')

        if dataset_name == 'dataset_1':
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')    
        else:
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
        
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.SAGEConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index, edge_weight)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index, edge_weight)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

##Config 12

In [ ]:
# ----------------------------- Configuration 12 --------------------------------
# -----------------------------                  --------------------------------
class config12(nn.Module):  
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(config12, self).__init__()

        self.conv1 = tg_nn.GraphConv(input_dim, hidden_dim, aggr='add')

        if dataset_name == 'dataset_1':
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')        
        else:
            self.conv2 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='add')
            
        self.conv3 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv4 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv5 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')
        self.conv6 = tg_nn.GraphConv(hidden_dim, hidden_dim, aggr='max')

        self.jk1 = JumpingKnowledge("lstm", hidden_dim, 3)
        self.jk2 = JumpingKnowledge("lstm", hidden_dim, 3)
        
        self.lin1 = torch.nn.Linear(hidden_dim, 63)
        self.lin2 = torch.nn.Linear(63, 3)
        
        self.active1 = nn.PReLU(hidden_dim)
        self.active2 = nn.PReLU(hidden_dim)
        self.active3 = nn.PReLU(hidden_dim)
        self.active4 = nn.PReLU(hidden_dim)
        self.active5 = nn.PReLU(hidden_dim)
        self.active6 = nn.PReLU(hidden_dim)
        self.active7 = nn.PReLU(63)

    def forward(self, data):
        x, edge_index, edge_weight, batch = data.x, data.edge_index, data.edge_attr ,data.batch
        edge_weight = 1 / edge_weight
        edge_weight = edge_weight.float()

        x = self.conv1(x, edge_index, edge_weight)
        x = self.active1(x)
        xs = [x]
        
        x = self.conv2(x, edge_index, edge_weight)
        x = self.active2(x)
        xs += [x]
        
        x = self.conv3(x, edge_index, edge_weight)
        x = self.active3(x)
        xs += [x]

        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk1(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.conv4(x, edge_index, edge_weight)
        x = self.active4(x)
        xs = [x]
       
        x = self.conv5(x, edge_index, edge_weight)
        x = self.active5(x)
        xs += [x]
        
        x =self.conv6(x, edge_index, edge_weight)
        x =  self.active6(x)
        xs += [x]
        
        # ~~~~~~~~~~~~Jumping knowledge applied ~~~~~~~~~~~~~~~
        x = self.jk2(xs)
        # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

        x = self.lin1(x)
        x = self.active7(x)
        x = F.dropout(x, p=0.1, training=self.training)
        x = self.lin2(x)

        return x

    def loss(self, pred, label):
      
        return (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum() 

        # return (pred - label).abs().sum()  #MAE
        # return F.mse_loss(pred, label)  #MSE 

################################################################################

# Training

In [ ]:
# Early Stopping
def early_stopping(val_loss, current_min):
    stop = False
    if val_loss[-1] > current_min:
        stop = True
    return stop

###############################################################################################

# Training 
def train(dataset, writer, dataset_name, config_selected):

    data_size = len(dataset)
    print(dataset_name)
# ------------------------------------------------------------------------------------------------------------------
    if dataset_name == 'dataset_1':
        print('Dataset 1 is being used.')
        loader = DataLoader(dataset[:int(data_size * 0.7)], batch_size=4, shuffle=True)
        validation_loader = DataLoader(dataset[int(data_size * 0.7): int(data_size * 0.9)], batch_size=1, shuffle=False)
        test_loader = DataLoader(dataset[int(data_size * 0.9):], batch_size=1, shuffle=False)
    else:
        print('Dataset 2 is being used.')
        loader = DataLoader(dataset[:int(data_size * 0.7)], batch_size=8, shuffle=True)
        validation_loader = DataLoader(dataset[int(data_size * 0.7): int(data_size * 0.9)], batch_size=1, shuffle=False)
        test_loader = DataLoader(dataset[int(data_size * 0.9):], batch_size=1, shuffle=False)      
# ------------------------------------------------------------------------------------------------------------------
    # MODIFICATION #1 Build the Model
    if config_selected == 'config1':
        model = config1(7, 112, 3)
        print('Configuration 1 is selected.')
    elif config_selected == 'config2':  
        model = config2(7, 112, 3)
        print('Configuration 2 is selected.')
    elif config_selected == 'config3':   
        model = config3(7, 112, 3)
        print('Configuration 3 is selected.')
    elif config_selected == 'config4':   
        model = config4(7, 112, 3)
        print('Configuration 4 is selected.')
    elif config_selected == 'config5':  
        model = config5(7, 112, 3) 
        print('Configuration 5 is selected.')
    elif config_selected == 'config6': 
        model = config6(7, 112, 3) 
        print('Configuration 6 is selected.')
    elif config_selected == 'config7': 
        model = config7(7, 112, 3)
        print('Configuration 7 is selected.')
    elif config_selected == 'config8':  
        model = config8(7, 112, 3)
        print('Configuration 8 is selected.') 
    elif config_selected == 'config9': 
        model = config9(7, 112, 3)
        print('Configuration 9 is selected.')
    elif config_selected == 'config10': 
        model = config10(7, 112, 3)
        print('Configuration 10 is selected.')
    elif config_selected == 'config11': 
        model = config11(7, 112, 3)
        print('Configuration 11 is selected.')
    elif config_selected == 'config12': 
        model = config12(7, 112, 3)
        print('Configuration 12 is selected.')
    else: 
        raise NotImplementedError
# ------------------------------------------------------------------------------------------------------------------
    # Move model to GPU
    model = model.to(device)
# ------------------------------------------------------------------------------------------------------------------
    opt = optim.AdamW(model.parameters(), lr=0.005)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(opt, mode='min', factor=0.1, patience=5, min_lr=0.00000001, verbose=True) 
# ------------------------------------------------------------------------------------------------------------------
    # print('parameters')
    # for name, param in model.named_parameters():
    #     if param.requires_grad:
    #         print (name, param.data)
    # print('----------------')
# ------------------------------------------------------------------------------------------------------------------
    # for plotting
    learning_curve_train = []
    learning_curve_val = []
# ------------------------------------------------------------------------------------------------------------------
    # for early stopping
    patience = 0
# ------------------------------------------------------------------------------------------------------------------    
    # MODIFICATION #2 Chossing the path to save the trained model
    if dataset_name == 'dataset_1':
        file_path = "Results_1/"
    else: 
        file_path = "Results_2/"
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    PATH = file_path + config_selected + '.pt'
    print(PATH)    
# ------------------------------------------------------------------------------------------------------------------
    learn_value_train = 0 
    learn_value_validation = 0
    # train
    for epoch in range(15000):

        lr = scheduler.optimizer.param_groups[0]['lr']
        print(lr)
     
        total_loss = 0
        total_len = 0

        model.train()
 
        for batch in loader:

            batch = batch.to(device)

            opt.zero_grad()
            pred = model(batch)
            label = batch.y         
            loss = model.loss(pred, label)
            loss = loss/ len(batch.y)
            loss.backward()
            opt.step()
            # total_loss += loss.item() # if loss isn't being averaged
            total_loss += loss.item() * len(batch.y) # if loss is being averaged
            total_len += len(label)

        total_loss /= total_len   #train loss
# ------------------------------------------------------------------------------------------------------------------
        # Validation
        val_loss = val(validation_loader, model)
        learning_curve_train.append(total_loss)
        learning_curve_val.append(val_loss)
        print("Epoch {}. Train Loss: {:.16f}. Validation Loss: {:.16f}".format(epoch, total_loss, val_loss))
# ------------------------------------------------------------------------------------------------------------------
        scheduler.step(val_loss)
# ------------------------------------------------------------------------------------------------------------------
        # Saving model and early stopping
        if epoch != 0:
            stop = early_stopping(learning_curve_val, current_min)
            if stop == False:
                current_min = learning_curve_val[-1]
                torch.save(model.state_dict(), PATH.format(epoch))                
                patience = 0
            else:
                patience = patience + 1
                print(patience)                
                if patience == 15:
                    break
        else:
            current_min = learning_curve_val[0]
# ------------------------------------------------------------------------------
    # MODIFICATION #3
    if config_selected == 'config1':
        final_model = config1(7, 112, 3)
        print('Configuration 1 best state loading...')
    elif config_selected == 'config2':  
        final_model = config2(7, 112, 3)
        print('Configuration 2 best state loading...')
    elif config_selected == 'config3':   
        final_model = config3(7, 112, 3)
        print('Configuration 3 best state loading...')
    elif config_selected == 'config4':   
        final_model = config4(7, 112, 3)
        print('Configuration 4 best state loading...')
    elif config_selected == 'config5':  
        final_model = config5(7, 112, 3) 
        print('Configuration 5 best state loading...')
    elif config_selected == 'config6': 
        final_model = config6(7, 112, 3) 
        print('Configuration 6 best state loading...')
    elif config_selected == 'config7': 
        final_model = config7(7, 112, 3)
        print('Configuration 7 best state loading...')
    elif config_selected == 'config8':  
        final_model = config8(7, 112, 3) 
        print('Configuration 8 best state loading...')
    elif config_selected == 'config9': 
        final_model = config9(7, 112, 3)
        print('Configuration 9 best state loading...')
    elif config_selected == 'config10': 
        final_model = config10(7, 112, 3)
        print('Configuration 10 best state loading...')
    elif config_selected == 'config11': 
        final_model = config11(7, 112, 3)
        print('Configuration 11 best state loading...')
    elif config_selected == 'config12': 
        final_model = config12(7, 112, 3)
        print('Configuration 12 best state loading...')
    else: 
        raise ValueError('The selected configuration does not exist.')
    # --------------------------------------------------------------------------
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    final_model.load_state_dict(torch.load(PATH))
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    # Final model to GPU
    final_model = final_model.to(device)
    # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    lowest_val_loss = val(validation_loader, final_model)
    test_loss = test(test_loader, final_model)

    print("Validation Loss: {:.16f}. Test Loss: {:.16f}".format(lowest_val_loss, test_loss))
    print('---------------')
# ------------------------------------------------------------------------------------------------------------------
    for j in range(len(learning_curve_train)):
        learn_value_train = learning_curve_train[j]
        learn_value_validation = learning_curve_val[j]
        print("{:.16f} {:.16f}".format(learn_value_train, learn_value_validation))

    return final_model

# Validation

In [ ]:
###############################################################################################
# Validation 

def val(loader, model):
    
    model.eval()

    error = 0
    total_loss = 0
    total_length = 0

    with torch.no_grad():
        for batch in loader:

            batch = batch.to(device)
        
            pred = model(batch)
            label = batch.y 
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~     
            # error = F.mse_loss(pred, label) # MSE
            # total_loss += error.item() * len(label) #MSE
            # ------------------------------------------------------------------
            # total_loss += (pred - label).abs().sum().item()  #MAE
            # ------------------------------------------------------------------
            total_loss += (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum().item()  # MAE for magnitude
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
            total_length += len(label)

    total_loss /= total_length

    return total_loss 

# Testing

In [ ]:
###############################################################################################
# Testing 

def test(loader, model):

    model.eval()

    error = 0
    total_loss = 0
    total_length = 0
         
    with torch.no_grad():

        for data in loader:

            data = data.to(device)

            pred = model(data)
            label = data.y    
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # error = F.mse_loss(pred, label) # MSE
            # total_loss += error.item() * len(label) #MSE
            # ------------------------------------------------------------------
            # total_loss += (pred - label).abs().sum().item()  #MAE
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            total_loss += (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum().item()  # MAE for magnitude
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
            total_length += len(label)

    total_loss /= total_length 

    return total_loss

#Loading Data 



In [ ]:
def get_data(pickled_file_path, dataset):
    with open(pickled_file_path, 'rb') as f:
        dataset_partial = torch.load(f)

        for i, data in enumerate(dataset_partial):
            feature = data.x
            edge_index = data.edge_index
            
            # print('---------------')
            # print(edge_index)
            # print(max(edge_index[0,:]))
            # print(max(edge_index[1,:]))
            # print('---------------')

            edge_values = data.edge_attr
            pos = data.pos
            y = data.y

            # create the data object
            data_obj = Data(x=feature, edge_index=edge_index, edge_attr=edge_values, pos=pos, y=y)
            dataset.append(data_obj)
      
    return dataset

In [ ]:
def load_data(dataset_name):

    dataset = []

    print(dataset_name + ' is being loaded...' )

    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_a_raw.pickle', dataset)
    print('1/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_b_raw.pickle', dataset)
    print('2/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_c_raw.pickle', dataset)
    print('3/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_d_raw.pickle', dataset)
    print('4/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_e_raw.pickle', dataset)
    print('5/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_f_raw.pickle', dataset)
    print('6/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_g_raw.pickle', dataset)
    print('7/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_h_raw.pickle', dataset)
    print('8/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_i_raw.pickle', dataset)
    print('9/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_j_raw.pickle', dataset)
    print('10/11 loaded.')
    dataset = get_data(dataset_name + '_pickle/' + dataset_name + '_k_raw.pickle', dataset)
    print('11/11 loaded.')

    print(len(dataset))
    print('Datasets have been concatenated.')
    
    return dataset

#Feature Preprocessing

In [ ]:
# Original Features:
# Physiacal Property | Fx | Fy | Fz
# New Features: 
# ------------------------------------------------------
# Physical Property | Fx | Fy | Fz | ro | phi | theta
# ro    = sqrt(Fx^2 + Fy^2 + Fz^2)
# phi   = cos^-1(Fz / ro)
# theta = sin^-1(Fy/ ro * sin(phi))
# ------------------------------------------------------

def add_new_features(dataset):

    len_dataset = len(dataset)

    for i, data in enumerate(dataset):

        ro = torch.sqrt((data.x[:,1]**2) + (data.x[:,2]**2)+ (data.x[:,3]**2)).unsqueeze(-1)
        phi = torch.acos(torch.div(data.x[:,3].unsqueeze(-1), ro))
        theta = torch.asin(torch.div(data.x[:,2].unsqueeze(-1), torch.mul(ro, torch.sin(phi))))
        data.x = torch.cat((data.x, ro, phi, theta), dim=1)
        data.x = torch.nan_to_num(data.x)
        
        print(data.x.shape)
        print(str(i+1)+ '/' +str(len_dataset)) 
    
    print('New features have been added.')
    return dataset

def mean_and_std_calc(dataset):

    len_dataset = len(dataset)
    
    feature = torch.empty(1,1)
    i = 0

    for i,data in enumerate(dataset):
        if i == 0:
            feature = data.x
        else:
            feature = torch.cat((feature, data.x), 0)
        i = i + 1
        
        print(str(i) + '/' + str(len_dataset))
        print(feature.shape)

    mean_val = torch.mean(feature, dim=0, keepdim=True)
    std_val = torch.std(feature, dim=0, keepdim=True)
    
    print(mean_val)
    print(std_val)
    
    return mean_val, std_val

def data_normalize(dataset):

    len_dataset = len(dataset)

    mean_val, std_val = mean_and_std_calc(dataset)

    for i,data in enumerate(dataset):
        data.x[:,1:] = (data.x[:,1:]-mean_val[:,1:])/std_val[:,1:]
        # data.x[:,:] = (data.x[:,:]-mean_val[:,:])/std_val[:,:]
        print(str(i+1) + '' + str(len_dataset))
    
    print('Data has been normalized.')
    return dataset

In [ ]:
def data_preprocessing(dataset):

    dataset = add_new_features(dataset)
    dataset = data_normalize(dataset)

    return dataset

# Final Run for Training

In [ ]:
random.seed(1)
torch.manual_seed(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
# ------------------------------------------------------------------------------
# Specifying the dataset
# dataset_name = 'dataset_1'
dataset_name = 'dataset_2'
# ----------------------------------------------------
# Load data
dataset = load_data(dataset_name)
# ----------------------------------------------------
# Preprocess data
dataset = data_preprocessing(dataset)
# # ----------------------------------------------------
random.Random(1).shuffle(dataset)
print('Pytorch Geometric dataset has been shuffeled.')

In [ ]:
# ----- Final Run ------

# config_selected = 'config1'     # 
# config_selected = 'config2'     # done
# config_selected = 'config3'     # 
# config_selected = 'config4'     # 
# config_selected = 'config5'     # 
# config_selected = 'config6'     # 
# config_selected = 'config7'     # 
# config_selected = 'config8'     # 
# config_selected = 'config9'     # 
# config_selected = 'config10'    # 
# config_selected = 'config11'    # 
# config_selected = 'config12'    # 

model = train(dataset, writer, dataset_name, config_selected)

# Reproducing the results

##Support Functions

In [ ]:
def val_reproduce(loader, model):
    
    model.eval()
    error = 0
    total_loss = 0
    total_length = 0

    actual = torch.empty(1,1)
    prediction = torch.empty(1,1)
    i = 0

    with torch.no_grad():
        for batch in loader:

            batch = batch.to(device)
        
            pred = model(batch)
            label = batch.y 
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~   
            # to save prediction and actual labels
            if i == 0:
                prediction = pred 
                actual = label
                i = i +1
            else:
                prediction = torch.cat((prediction, pred), 0)
                actual = torch.cat((actual, label), 0)
                i = i +1
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~      
            # error = F.mse_loss(pred, label) # MSE
            # total_loss += error.item() * len(label) #MSE
            # ------------------------------------------------------------------
            # total_loss += (pred - label).abs().sum().item()  #MAE
            # ------------------------------------------------------------------
            total_loss += (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum().item()  # MAE for magnitude
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~  
            total_length += len(label)

    total_loss /= total_length

    return total_loss, prediction, actual  

In [ ]:
def test_reproduce(loader, model):

    model.eval()
    error = 0
    total_loss = 0
    total_length = 0

    actual = torch.empty(1,1)
    prediction = torch.empty(1,1)
    i = 0
         
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            pred = model(data)
            label = data.y  
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # to save prediction and actual labels
            if i == 0:
                prediction = pred 
                actual = label
                i = i +1
            else:
                prediction = torch.cat((prediction, pred), 0)
                actual = torch.cat((actual, label), 0)
                i = i +1
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            # error = F.mse_loss(pred, label) # MSE
            # total_loss += error.item() * len(label) #MSE
            # ------------------------------------------------------------------
            # total_loss += (pred - label).abs().sum().item()  #MAE
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
            total_loss += (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))).sum().item()  # MAE for magnitude
            # ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~ 
            total_length += len(label)

    total_loss /= total_length 

    return total_loss, prediction, actual

In [ ]:
def max_magnitude_error(loader, model):

    model.eval()
    error = 0
    max_errors = []
    running_time = []
    max_displacement = []
         
    with torch.no_grad():
        for data in loader:
            data = data.to(device)
            
            t0 = time.time()
            pred = model(data)
            label = data.y  
            t1 = time.time()
            total = t1-t0

            running_time.append(total)

            loss = (torch.sqrt(((pred[:,0] - label[:,0])**2).unsqueeze(-1) + ((pred[:,1] - label[:,1])**2).unsqueeze(-1) + ((pred[:,2] - label[:,2])**2).unsqueeze(-1))) # MAE for magnitude

            max_loss = max(loss).item()
            print(max_loss)
            max_errors.append(max_loss)

            displacement = torch.sqrt((label[:,0]**2).unsqueeze(-1) + (label[:,1]**2).unsqueeze(-1) + (label[:,2]**2).unsqueeze(-1))
            max_displacement.append(max(displacement).item())

    return max_displacement, max_errors, running_time

In [ ]:
def reproduce(dataset, writer, config_selected, save, mean_mag_results, max_error_results):

    data_size = len(dataset)
    # -----------------------------------------------------------------------------------------------
    if mean_mag_results == 1 and max_error_results == 0:
        validation_loader = DataLoader(dataset[int(data_size * 0.7): int(data_size * 0.9)], batch_size=8, shuffle=False)
        test_loader = DataLoader(dataset[int(data_size * 0.9):], batch_size=8, shuffle=False)
    else:
        validation_loader = DataLoader(dataset[int(data_size * 0.7): int(data_size * 0.9)], batch_size=1, shuffle=False)
        test_loader = DataLoader(dataset[int(data_size * 0.9):], batch_size=1, shuffle=False)
    # ------------------------------------------------------------------------------------------------------------------
    # Select the path to the file which holds the final model
    if dataset_name == 'dataset_1':
        file_path = "Results_1/"
    else:
        file_path = "Results_2/"
    # --------------------------------------------
    PATH = file_path + config_selected + '.pt'
    print(PATH)
    # --------------------------------------------
    # Initialize the model
    if config_selected == 'config1': 
        final_model = config1(7, 112, 3)
        print('Configuration 1 has been selected.')
    elif config_selected == 'config2':  
        final_model = config2(7, 112, 3)
        print('Configuration 2 has been selected.')
    elif config_selected == 'config3':
        final_model = config3(7, 112, 3) 
        print('Configuration 3 has been selected.')   
    elif config_selected == 'config4':
        final_model = config4(7, 112, 3) 
        print('Configuration 4 has been selected.')  
    elif config_selected == 'config5':
        final_model = config5(7, 112, 3)
        print('Configuration 5 has been selected.')
    elif config_selected == 'config6': 
        final_model = config6(7, 112, 3)
        print('Configuration 6 has been selected.')
    elif config_selected == 'config7': 
        final_model = config7(7, 112, 3)
        print('Configuration 7 has been selected.')
    elif config_selected == 'config8':    
        final_model = config8(7, 112, 3)
        print('Configuration 8 has been selected.')
    elif config_selected == 'config9':        
        final_model = config9(7, 112, 3)  
        print('Configuration 9 has been selected.')  
    elif config_selected == 'config10':    
        final_model = config10(7, 112, 3)
        print('Configuration 10 has been selected.')   
    elif config_selected == 'config11':    
        final_model = config11(7, 112, 3) 
        print('Configuration 11 has been selected.')  
    elif config_selected == 'config12':    
        final_model = config12(7, 112, 3)
        print('Configuration 12 has been selected.')
    else:  
        raise NotImplementedError            
    # ----------------------------------------------
    # Load the model
    final_model.load_state_dict(torch.load(PATH))
    # ----------------------------------------------
    # Final model to GPU
    final_model = final_model.to(device)
    # ----------------------------------------------
    # ######################################################################################################
    # Reproducing the mean of maximum errors in magnitude and runtime results
    if max_error_results == 1:

        max_displacement, max_error_test, running_time = max_magnitude_error(test_loader, final_model)

        max_maximum_displacement = max(max_displacement)
        print(max_maximum_displacement)
        mean_maximum_displacement = statistics.mean(max_displacement)
        print(mean_maximum_displacement)
        std_max_displacement = statistics.stdev(max_displacement)
        print(std_max_displacement)
        print("Maximum displacement in the test set: {:.4f}. Average maximum displacement {:.4f} +/- {:.4f}".format(max_maximum_displacement, mean_maximum_displacement, std_max_displacement))

        mean_maximum_test_loss = statistics.mean(max_error_test)
        std_maximum_test_loss = statistics.stdev(max_error_test)
        print("Average maximum magnitude error: {:.4f} +/- {:.4f}".format(mean_maximum_test_loss, std_maximum_test_loss))

        mean_time = statistics.mean(running_time)
        std_time = statistics.stdev(running_time)
        print("Average running time: {:.4f} +/- {:.4f}".format(mean_time, std_time))

    # ######################################################################################################
    # Reproducing the results in Table 1 and Table 2
    if mean_mag_results == 1:
    
        if save == 0:
            lowest_val_loss = val(validation_loader, final_model)
            test_mse = test(test_loader, final_model)
            print("Validation Loss: {:.16f} Test Loss: {:.16f}".format(lowest_val_loss, test_mse))
        else:
            lowest_val_loss, prediction_val, actual_val = val_reproduce(validation_loader, final_model)
            test_mse, prediction_test, actual_test = test_reproduce(test_loader, final_model)
            print("Validation Loss: {:.16f} Test Loss: {:.16f}".format(lowest_val_loss, test_mse))

            # -----------------------------------------------------------------------------------------------
            # Saving the results for further analysis 
            print('Saving the results...')
            
            # Validation set 
            prediction_val = prediction_val.cpu()
            actual_val = actual_val.cpu()
            prediction_val = prediction_val.numpy()
            prediction_val_df = pd.DataFrame(prediction_val)
            # ------------------------------------------------------------------------------------------------
            prediction_val_df.to_csv(file_path + 'csv/val/prediction_' + config_selected + '.csv')
            # ------------------------------------------------------------------------------------------------
            actual_val = actual_val.numpy()
            actual_val_df = pd.DataFrame(actual_val)
            # ------------------------------------------------------------------------------------------------
            actual_val_df.to_csv(file_path + 'csv/val/actual_' + config_selected + '.csv')    
            # ------------------------------------------------------------------------------------------------
            
            # Test set 
            prediction_test = prediction_test.cpu()
            actual_test = actual_test.cpu()
            prediction_test = prediction_test.numpy()
            prediction_test_df = pd.DataFrame(prediction_test)
            # ------------------------------------------------------------------------------------------------
            prediction_test_df.to_csv(file_path + 'csv/test/prediction_' + config_selected + '.csv')        
            # ------------------------------------------------------------------------------------------------
            actual_test = actual_test.numpy()
            actual_test_df = pd.DataFrame(actual_test)
            # ------------------------------------------------------------------------------------------------    
            actual_test_df.to_csv(file_path + 'csv/test/actual_' + config_selected + '.csv')                
            # ------------------------------------------------------------------------------------------------
      

##Final Run

In [ ]:
random.seed(1)
torch.manual_seed(1)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('CUDA availability:', torch.cuda.is_available())
writer = SummaryWriter("./log/" + datetime.now().strftime("%Y%m%d-%H%M%S"))
# ------------------------------------------------------------------------------
# Specifying the dataset
# dataset_name = 'dataset_1'
dataset_name = 'dataset_2'
# ----------------------------------------------------
# Load data
dataset = load_data(dataset_name)
# ----------------------------------------------------
# Preprocess data
dataset = data_preprocessing(dataset)
# # ----------------------------------------------------
random.Random(1).shuffle(dataset)
print('Pytorch Geometric dataset has been shuffeled.')

In [ ]:
# If save is equal to 1, the prediction and actual values will be saved for 
# further result processing.
save = 0 
mean_mag_results = 0
max_error_results = 1
# ----------------------------------------------------------------------------------------------------
# Please note:
# Best performing model for dataset 1 is config 9
# Best performing model for dataset 2 is config 3
# ----------------------------------------------------------------------------------------------------
# Select configuration

# reproduce(dataset, writer, 'config1', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config2', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config3', save, mean_mag_results, max_error_results) # best performance dataset 2
# reproduce(dataset, writer, 'config4', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config5', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config6', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config7', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config8', save, mean_mag_results, max_error_results)
reproduce(dataset, writer, 'config9', save, mean_mag_results, max_error_results) # best performance dataset 1
# reproduce(dataset, writer, 'config10', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config11', save, mean_mag_results, max_error_results)
# reproduce(dataset, writer, 'config12', save, mean_mag_results, max_error_results)